The norm $\text{nm}_H^G \colon A(H) \to A(G)$ the Burnside Tambara functor is given by 
$$
    \text{nm}_H^G(X) = \text{Set}^H(G,X),
$$
where the right-hand side is the $H$-equivariant functions from $G$ to $X$. This becomes a $G$-set via $$(g \cdot f)(x) = f(xg).$$ 

We can also encode these norms in the table of marks. For an arbitrary class function $f \colon \text{Sub}(G)/G \to \mathbb{Z}$, 
$$
    \text{nm}_H^G(f)([K]) = \prod_{KgH \in K \backslash G / H} f([K^g \cap H]).
$$

In [1]:
# A function to compute the norm from H to G in the Burnside ring
#
# INPUT
# H, G are GAP groups
# f is a vector defining a linear combination of H-orbits
#   The order of orbits is the same as the rows of the table of marks of H, as given by GAP
#   For example, if H is the quaternion group Q_8, then the vector f = (0,1,-1,0,2,0)
#   represents the element Q_8/C_2 - Q_8/I + 2 Q_8/K, where I = <i> and K = <k>. 
#   
#   For Q_8, the GAP table of marks presents conjugacy classes of subgroups in the order 
#   e, C_2, I, J, K, Q_8, where I = <i>, J = <j>, and K = <k>. 
#
# OUTPUT 
# a vector n encoding a linear combination of G-orbits, with order of orbits as above
#
# EXAMPLES 
#  tbd
#
def norm(H,G,f):
    # if H is not a subgroup of G, then throw an error 
    if not gap.IsSubgroup(G,H): 
        raise ValueError("You fool! H must be a subgroup of G") # maybe the error could be nicer

    tomG = G.TableOfMarks() # create the table of marks of G
    nG = len(G.ConjugacyClassesSubgroups()) # dimension of table of marks
    matG = matrix(ZZ, tomG.MatTom()) # table of marks as a matrix 
    repsG = [tomG.RepresentativeTom(n+1) for n in range(nG)] # reps of conjugacy classes

    tomH = H.TableOfMarks() # create the table of marks of H
    nH = len(H.ConjugacyClassesSubgroups()) # dimension of table of marks
    matH = matrix(ZZ, tomH.MatTom()) # table of marks as a matrix
    repsH = [tomH.RepresentativeTom(n+1) for n in range(nH)] # reps of conjugacy classes

    marksf = f * matH # convert the vector f of coefficients into an element of the ghost ring
    nm = vector(ZZ, [1]*nG) # empty product is 1; setup for looping
    for i in range(nG):
        K = repsG[i] # we're finding the component indexed by K
        for(g,_) in G.DoubleCosetRepsAndSizes(K,H): # only need the reps of double cosets
            HcapKg = gap.Intersection(H,K.ConjugateGroup(g)) # form the intersection of H and the conjugate of K by g
            index = next(j for j,r in enumerate(repsH) if H.IsConjugate(r, HcapKg)) # find the index of the subgroup of H that represents H \cap K^g
            nm[i] *= marksf[index] # multiply the [K] component of the norm by marksf([H \cap K^g])
    return matG.solve_left(nm) # convert back into a list of coefficients

In [2]:
S5 = gap("SymmetricGroup(5)")
S4 = gap("SymmetricGroup(4)")
C2 = gap("SymmetricGroup(2)")
e = gap("SymmetricGroup(1)")
# norm from e to C2 of 0 is 0 and of 1 is 1
# in general norm from e to C2 of a positive a is (a choose 2, a)
assert norm(e,C2,vector([0])) == vector([0,0])
assert norm(e,C2,vector([1])) == vector([0,1])
assert norm(e,C2,vector([4])) == vector([6,4])

# norm from e to S5 of 0 is 0 and of 1 is 1
# these are slow, comment out if you don't want to wait 4s
assert norm(e,S5,vector([0])) == vector(ZZ,[0] * 19)
assert norm(e,S5,vector([1])) == vector(ZZ, 19, {18:1})

norm(S4, S5, vector(ZZ, 11, {3:1,4:1}))

(4341, 0, 264, 11, 14, 0, 0, 2, 0, 0, 6, 0, 3, 0, 0, 0, 0, 0, 0)